In [1]:
# !pip install langdetect

In [ ]:
import os
import glob
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

def separar_sms_espanol(directorio_dataset, archivo_salida="sms_espanol.txt"):
    # Obtener todos los archivos .txt en el directorio
    archivos_txt = glob.glob(os.path.join(directorio_dataset, "*.txt"))
    
    total_sms = 0
    sms_espanol = 0
    errores_deteccion = 0
    
    # Crear archivo para guardar SMS en español
    with open(archivo_salida, 'w', encoding='utf-8') as archivo_esp:
        for archivo in archivos_txt:
            print(f"Procesando: {os.path.basename(archivo)}")
            
            with open(archivo, 'r', encoding='utf-8', errors='ignore') as f:
                for linea_num, linea in enumerate(f, 1):
                    linea = linea.strip()
                    
                    # Saltar líneas vacías
                    if not linea:
                        continue
                    
                    total_sms += 1
                    
                    try:
                        # Detectar idioma
                        idioma = detect(linea)
                        if idioma == 'es':
                            sms_espanol += 1
                            # Escribir SMS en español al archivo
                            archivo_esp.write(linea + '\n')
                    except LangDetectException:
                        # Si no se puede detectar el idioma
                        errores_deteccion += 1
    
    return total_sms, sms_espanol, errores_deteccion

# Ejecutar análisis y separar SMS en español
directorio = "dataset/tweet_sms"
archivo_salida = "dataset_2/sms_solo_español.txt"

total, espanol, errores = separar_sms_espanol(directorio, archivo_salida)

print(f"\n=== RESULTADOS ===")
print(f"Total de SMS analizados: {total}")
print(f"SMS en español: {espanol}")
print(f"Porcentaje en español: {(espanol/total)*100:.2f}%")
print(f"Errores de detección: {errores}")
print(f"SMS en otros idiomas: {total - espanol - errores}")
print(f"\n✅ SMS en español guardados en: {archivo_salida}")

Procesando: tweet_sms_2018-02.txt
Procesando: tweet_sms_2020-06.txt
Procesando: tweet_sms_2020-12.txt
Procesando: tweet_sms_2022-03.txt
Procesando: tweet_sms_2022-02.txt
Procesando: tweet_sms_2020-07.txt
Procesando: tweet_sms_2018-03.txt
Procesando: tweet_sms_2018-01.txt
Procesando: tweet_sms_2020-11.txt
Procesando: tweet_sms_2020-05.txt
Procesando: tweet_sms_2022-01.txt
Procesando: tweet_sms_2020-04.txt
Procesando: tweet_sms_2020-10.txt
Procesando: tweet_sms_2018-10.txt
Procesando: tweet_sms_2018-04.txt
Procesando: tweet_sms_2022-05.txt
Procesando: tweet_sms_2022-04.txt
Procesando: tweet_sms_2020-01.txt
Procesando: tweet_sms_2018-05.txt
Procesando: tweet_sms_2018-11.txt
Procesando: tweet_sms_2018-07.txt
Procesando: tweet_sms_2020-03.txt
Procesando: tweet_sms_2022-06.txt
Procesando: tweet_sms_2022-07.txt
Procesando: tweet_sms_2020-02.txt
Procesando: tweet_sms_2018-12.txt
Procesando: tweet_sms_2018-06.txt
Procesando: tweet_sms_2019-09.txt
Procesando: tweet_sms_2019-08.txt
Procesando: tw

In [ ]:
# !pip install groq
# !pip install ollama

In [21]:
import ollama
import os

def crear_directorio(directorio):
    """Crea el directorio si no existe"""
    if not os.path.exists(directorio):
        os.makedirs(directorio)

def es_spam_adicional(sms):
    """Verificación adicional para casos que el modelo puede pasar por alto"""
    sms_lower = sms.lower()
    
    # Patrones específicos de SPAM
    patrones_spam = [
        # Phishing bancario con enlaces
        ("banco estado" in sms_lower and "http" in sms_lower),
        ("bbva" in sms_lower and "bit.ly" in sms_lower),
        ("santander" in sms_lower and "http" in sms_lower),
        
        # Sorteos y rankings falsos
        ("ranking" in sms_lower and "http" in sms_lower),
        ("sorteo" in sms_lower and "http" in sms_lower),
        ("has ganado" in sms_lower),
        
        # Publicidad de productos
        ("adelgazante" in sms_lower and "viral" in sms_lower),
        ("unidades limitadas" in sms_lower),
        
        # Enlaces acortados sospechosos
        ("bit.ly" in sms_lower and any(x in sms_lower for x in ["banco", "cuenta", "tarjeta"])),
        ("cutt.ly" in sms_lower and any(x in sms_lower for x in ["banco", "cuenta", "tarjeta"])),
        
        # Paquetes falsos
        ("paquete" in sms_lower and "http" in sms_lower and "direccion" in sms_lower),
        
        # Ofertas comerciales agresivas
        ("jazztel" in sms_lower and "mioferta" in sms_lower),
        
        # NUEVOS PATRONES AÑADIDOS:
        # Servicios técnicos no solicitados
        ("problemas de internet" in sms_lower and any(tel in sms_lower for tel in ["099", "24807739", "telefono"])),
        ("videporteros" in sms_lower and "camaras" in sms_lower),
        ("alarmas" in sms_lower and "respuesta celular" in sms_lower),
        ("amc.com.uy" in sms_lower),
        
        # Publicidad de servicios profesionales
        ("abogados" in sms_lower and "whatsapp" in sms_lower),
        ("lawservice.com" in sms_lower),
    ]
    
    return any(patron for patron in patrones_spam)

def clasificar_sms_spam(archivo_entrada, directorio_salida="dataset_3", limite=None):
    """
    Clasifica SMS como spam o no spam usando Ollama con verificación adicional
    """
    # Crear directorio de salida
    crear_directorio(directorio_salida)
    
    # Rutas de archivos de salida
    archivo_spam = os.path.join(directorio_salida, "spam.txt")
    archivo_no_spam = os.path.join(directorio_salida, "no_spam.txt")
    
    # Leer SMS del archivo
    with open(archivo_entrada, 'r', encoding='utf-8') as f:
        sms_list = [linea.strip() for linea in f if linea.strip()]
    
    # Aplicar límite si se especifica
    if limite:
        sms_list = sms_list[:limite]
        print(f"Procesando {len(sms_list)} SMS (límite: {limite})")
    else:
        print(f"Procesando todos los SMS: {len(sms_list)}")
    
    spam_count = 0
    no_spam_count = 0
    
    # Prompt mejorado
    prompt_template = """Analiza este SMS y determina si es SPAM o NO SPAM. Sé MUY ESTRICTO.

🚨 CRITERIOS PARA SPAM (CUALQUIERA DE ESTOS ES SPAM):
- CUALQUIER enlace pidiendo datos bancarios o personales
- Mensajes de bancos pidiendo "verificar", "activar" o "actualizar" cuentas
- Enlaces sospechosos de empresas bancarias (especialmente con dominios raros)
- Sorteos, premios o "has ganado" no solicitados
- Phishing bancario (BBVA, Santander, BancoEstado, Davivienda, etc.)
- Ofertas de trabajo con salarios irreales
- Urgencia falsa ("cuenta será bloqueada", "suspendida")
- PUBLICIDAD COMERCIAL NO SOLICITADA (productos adelgazantes, servicios técnicos, etc.)
- Mensajes promocionales con WhatsApp o teléfonos
- Ofertas de servicios profesionales (abogados, técnicos, internet, cámaras)
- Mensajes de "paquetes detenidos" pidiendo pagos o datos
- Enlaces acortados sospechosos (bit.ly, cutt.ly con contenido bancario)
- Mensajes pidiendo "confirmar", "verificar" o "actualizar" datos
- Mensajes de "rankings" o "sorteos" con enlaces sospechosos
- SERVICIOS TÉCNICOS NO SOLICITADOS (internet, videoporteros, alarmas)

✅ CRITERIOS PARA NO SPAM (SOLO ESTOS):
- Notificaciones reales de servicios ya contratados SIN enlaces sospechosos
- Información educativa pura (sin enlaces comerciales)
- Alertas de seguridad que NO piden datos ni tienen enlaces
- Mensajes informativos oficiales sin solicitar acciones
- INFORMACIÓN DE SALUD PÚBLICA OFICIAL
- NOTICIAS OFICIALES DE GOBIERNO

⚠️ ESPECIAL ATENCIÓN: 
- Si un mensaje menciona bancos Y tiene enlaces, ES SPAM
- Si menciona "ranking", "sorteo" con enlaces, ES SPAM
- Si es publicidad de productos o servicios, ES SPAM
- Si ofrece servicios técnicos con teléfonos, ES SPAM

SMS: "{sms}"

Responde SOLO "SPAM" o "NO_SPAM"."""

    # Abrir archivos de salida
    with open(archivo_spam, 'w', encoding='utf-8') as f_spam, \
         open(archivo_no_spam, 'w', encoding='utf-8') as f_no_spam:
        
        for i, sms in enumerate(sms_list, 1):
            print(f"Procesando SMS {i}/{len(sms_list)}: {sms[:50]}...")
            
            try:
                # Verificación adicional primero
                if es_spam_adicional(sms):
                    spam_count += 1
                    f_spam.write(sms + '\n')
                    print(f"  → SPAM (detectado por patrones)")
                    continue
                
                # Consultar al modelo
                respuesta = ollama.generate(
                    model='llama3.1:8b',
                    prompt=prompt_template.format(sms=sms),
                    stream=False,
                    options={
                        'num_ctx': 4096,        # Contexto más grande
                        'num_batch': 512,       # Lotes más grandes
                        'num_gpu': 1,           # Forzar GPU
                        'num_thread': 0,        # Usar todos los threads
                    } 
                )
                
                # Clasificar y guardar
                respuesta_texto = respuesta['response'].upper().strip()
                
                if "SPAM" in respuesta_texto and "NO_SPAM" not in respuesta_texto:
                    spam_count += 1
                    f_spam.write(sms + '\n')
                    print(f"  → SPAM")
                else:
                    no_spam_count += 1
                    f_no_spam.write(sms + '\n')
                    print(f"  → NO SPAM")
                    
            except Exception as e:
                print(f"  → ERROR: {e}")
                no_spam_count += 1
                f_no_spam.write(sms + '\n')
    
    # Mostrar resultados
    print(f"\n=== RESULTADOS ===")
    print(f"Total de SMS analizados: {len(sms_list)}")
    print(f"SMS clasificados como SPAM: {spam_count}")
    print(f"SMS clasificados como NO SPAM: {no_spam_count}")
    print(f"Porcentaje de SPAM: {(spam_count/len(sms_list)*100):.1f}%")
    print(f"\n✅ Archivos guardados en:")
    print(f"  - SPAM: {archivo_spam}")
    print(f"  - NO SPAM: {archivo_no_spam}")

# Ejecutar con verificación mejorada
clasificar_sms_spam("dataset_2/sms_solo_español.txt")

Procesando todos los SMS: 2676
Procesando SMS 1/2676: Son los malware más peligrosos difíciles de detect...
  → NO SPAM
Procesando SMS 2/2676: WWW.CLASI.CL Recarga tu Celular en WWW.CLASI.CL Pa...
  → SPAM
Procesando SMS 3/2676: El verano no se acaba Ingresa VERANOCABIFY y obten...
  → SPAM
Procesando SMS 4/2676: VF Info Este San Valentin Vodafone te regala gigas...
  → SPAM
Procesando SMS 5/2676: Regalale flores para San Valentin Usa el codigo VA...
  → SPAM
Procesando SMS 6/2676: VF Publi Con la App Mi Vodafone vive el amor todos...
  → SPAM
Procesando SMS 7/2676: VF Info Este San Valentin por ser cliente Vodafone...
  → NO SPAM
Procesando SMS 8/2676: VE INTO Desde anora deberas usar la App Mi Vodafon...
  → NO SPAM
Procesando SMS 9/2676: Regalale flores para San Valentin Usa el codigo VA...
  → SPAM
Procesando SMS 10/2676: Busca asesoramiento legal Abogados en todas la esp...
  → SPAM (detectado por patrones)
Procesando SMS 11/2676: Regalale flores para San Valentin Usa el codigo VA

In [29]:
import ollama
import os

def crear_directorio(directorio):
    """Crea el directorio si no existe"""
    if not os.path.exists(directorio):
        os.makedirs(directorio)

def limpiar_sms(texto):
    """Limpia el SMS eliminando comillas y caracteres no deseados"""
    # Eliminar comillas dobles y simples del inicio y fin
    texto = texto.strip().strip('"').strip("'")
    # Eliminar comillas dobles internas si están al inicio y fin
    if texto.startswith('"') and texto.endswith('"'):
        texto = texto[1:-1]
    if texto.startswith("'") and texto.endswith("'"):
        texto = texto[1:-1]
    return texto.strip()

def generar_sms_no_spam(cantidad=100):
    """
    Genera SMS de NO SPAM en contexto peruano usando Ollama
    """
    crear_directorio("dataset_4")
    archivo_salida = "dataset_4/sms_no_spam_peru.txt"
    
    # Prompt mejorado con ejemplos reales
    prompt = """Genera un SMS legítimo (NO SPAM) en contexto peruano. Debe ser:

📱 TIPOS DE SMS LEGÍTIMOS PERUANOS:
- Notificaciones bancarias reales (Caja Arequipa, BCP, Interbank, Scotiabank, BBVA, INTERBANK) SIN enlaces
- Alertas de transacciones bancarias exitosas
- Códigos de verificación de apps peruanas (Yape y Plin)
- Recordatorios de citas médicas (EsSalud, clínicas)
- 
- Información educativa del MINSA sobre salud
- Alertas del SENAMHI sobre clima
- Notificaciones de servicios públicos (luz (Electro Puno, etc), agua (Sedapal), gas)
- Mensajes informativos del gobierno peruano
- Confirmaciones de compras en tiendas conocidas
- Recordatorios de pagos de servicios

🇵🇪 CONTEXTO PERUANO:
- Usar empresas peruanas conocidas
- Mencionar lugares de Perú
- Usar terminología local
- NO incluir enlaces sospechosos
- NO solicitar datos personales
- NO usar comillas en la respuesta

EJEMPLOS DE SMS LEGALES PERUANOS:
1. [CENTRO DE SALUD] te invita a una consulta programada por Video. Datos del turno: Dr(a): [NOMBRE]] a las [TIME]] [URL]
2. Felicidades! Activaste tu paquete de 100MB x 1 dia. Conoce mas beneficios en el App Mi Movistar o aquí www.smvst.com/recargapre
3. Por tu seguridad, no contrates lineas moviles(chips) en la calle. Pueden usar tus datos personales y huella dactilar para involucrarte en fraudes o mas delitos.
4. Si al contratar, migrar o portar tu numero, el operador te ofrece alguna oferta o promoción, verifica que se encuentre indicada en tu contrato.
5. Hola su envio de Mercado Libre con numero de seguimiento (25/1955253) fue asignado a reparto el dia de hoy. [AGENCIA DE ENVIO PERUANO].
6. Hola [NOMBRE], [BANCO PERUANO] te informa sobre la operación TIN LINEA A CELULAR por [monto en soles] hoy a las [time] realizada en CAJA MOVIL.
7. Has solicitado un codigo de validación para ingresar a [APPS PERUANAS]. EVITA LOS FRAUDES, NO lo compartas con nadie y usalo solo en tu APP. Codigo: [CODIGO].
8. [CODIGO DE GOOGLE]: codigo de verificación de Google. No lo compartas.
9. Hola, recuerda pagar tu recibo Movil de [monto en soles] que vence [TIME] x la app MiMovistar, Yape o app de banco. Evita la suspensión y cobro de reconexion de [monto en soles].

Responde SOLO con el texto del SMS, sin comillas, sin explicaciones adicionales (máximo 160 caracteres):"""

    print(f"🚀 Generando {cantidad} SMS de NO SPAM peruanos...")
    
    with open(archivo_salida, 'w', encoding='utf-8') as archivo:
        for i in range(cantidad):
            try:
                respuesta = ollama.generate(
                    model='llama3.1:8b',
                    prompt=prompt,
                    stream=False,
                    options={
                        'temperature': 0.8,      # Más creatividad
                        'num_ctx': 4096,         # Contexto más grande
                        'num_batch': 512,        # Lotes más grandes
                        'num_gpu': 1,            # Forzar GPU
                        'num_thread': 0,         # Usar todos los threads
                    }
                )
                
                sms_crudo = respuesta['response'].strip()
                sms_limpio = limpiar_sms(sms_crudo)
                
                # Solo escribir si el SMS tiene contenido válido
                if sms_limpio and len(sms_limpio) > 10:
                    archivo.write(sms_limpio + '\n')
                    print(f"{i+1:3d}. {sms_limpio}")
                else:
                    print(f"{i+1:3d}. DESCARTADO: {sms_crudo}")
                    
            except Exception as e:
                print(f"{i+1:3d}. ERROR: {e}")
    
    print(f"\n✅ SMS generados guardados en: {archivo_salida}")


print("\n" + "="*50 + "\n")

# Generar nuevos SMS limpios
generar_sms_no_spam(2000)



🚀 Generando 2000 SMS de NO SPAM peruanos...
  1. Tu recibo de luz de Electro Puno vence el proximo dia 15, puedes pagar en la app Mi Movistar o directamente en nuestra oficina.
  2. ¡Atención! El SENAMHI te informa que se prevé una tormenta eléctrica en Lima para mañana a las 10:00 am. Evita áreas de riesgo".
  3. Hasta el próximo viernes tienes un descuento del 10% en tu cuenta Scotiabank. Utiliza el código: SB1009 al pagar tus compras a través de Yape o la App del Banco. No lo compartas con nadie.
  4. Has solicitado un codigo de validación para ingresar a Yape. EVITA LOS FRAUDES, NO lo compartas con nadie y usalo solo en tu APP. Codigo: 1234
  5. Te informamos que tu cuenta de agua Sedapal ha sido pagada hasta el día 15/08. Gracias por pago a tiempo.
  6. Recuerda pagar tu recibo Movil de 50 soles que vence mañana x la app MiMovistar.
  7. Felicidades! Activaste tu paquete de 100MB x 1 dia. Conoce más beneficios en la app Mi Movistar.
  8. ¡Felicidades! Has aprobado el préstamo de

In [28]:
# Eliminar los duplicados del archivo .txt
def eliminar_duplicados(archivo_entrada, archivo_salida):
    """Elimina líneas duplicadas de un archivo de texto"""
    # Convertir a rutas absolutas para mostrar la ubicación completa
    ruta_entrada = os.path.abspath(archivo_entrada)
    ruta_salida = os.path.abspath(archivo_salida)
    
    print(f"🔍 Procesando archivo: {ruta_entrada}")
    
    # Contador de líneas procesadas y eliminadas
    total_lineas = 0
    duplicados = 0
    lineas_vistas = set()
    
    with open(archivo_entrada, 'r', encoding='utf-8') as f_in, \
         open(archivo_salida, 'w', encoding='utf-8') as f_out:
        for linea in f_in:
            linea = linea.strip()
            total_lineas += 1
            
            if not linea:
                continue
                
            if linea in lineas_vistas:
                duplicados += 1
            else:
                f_out.write(linea + '\n')
                lineas_vistas.add(linea)
    
    print(f"✅ Duplicados eliminados:")
    print(f"   - Archivo de entrada: {ruta_entrada}")
    print(f"   - Archivo de salida: {ruta_salida}")
    print(f"   - Total líneas procesadas: {total_lineas}")
    print(f"   - Líneas únicas: {len(lineas_vistas)}")
    print(f"   - Duplicados eliminados: {duplicados}")

# Ejemplo de uso:
eliminar_duplicados("dataset_4/sms_no_spam_peru.txt", "dataset_4/sms_no_spam_peru_clean.txt")

🔍 Procesando archivo: /Users/franspaxi/machine-learning/dataset_4/sms_no_spam_peru.txt
✅ Duplicados eliminados:
   - Archivo de entrada: /Users/franspaxi/machine-learning/dataset_4/sms_no_spam_peru.txt
   - Archivo de salida: /Users/franspaxi/machine-learning/dataset_4/sms_no_spam_peru_clean.txt
   - Total líneas procesadas: 50
   - Líneas únicas: 50
   - Duplicados eliminados: 0
